# Network component analysis

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import networkx as nx
from networkx.algorithms import community
from tqdm import tqdm

Path.mkdir(Path("analysis"), exist_ok=True)

In [13]:
edgelists = []
nx_graphs = []

for file in Path("edgelistsSample").iterdir():
    tmp_edgelist = pd.read_csv(file)
    edgelists.append(tmp_edgelist)

    tmp_nx_graph = nx.from_pandas_edgelist(tmp_edgelist, source='Source', target='Target', create_using=nx.Graph())
    nx.set_edge_attributes(tmp_nx_graph, 1, 'weight')
    nx_graphs.append(tmp_nx_graph)

In [15]:
louvain_nx = community.louvain_partitions(nx_graphs[0])
first = next(louvain_nx)
print(f"After first passage: {len(first)} communities")
second = next(louvain_nx)
print(f"After second passage: {len(second)} communities")

After first passage: 25 communities


StopIteration: 

In [16]:
# Louvain implementation
def louvain(G, npassage):
    # Will contain the graph and the communities after each passage
    data = {}
    for i in range(npassage):
        print(f"Passage {i}", flush=True)
        old_G, G, communities, get_community = louvain_step(G, G, i==0)
        data[i] = (old_G, communities, get_community)
        print(f"There are {len(communities)} communities after passage {i}", flush=True)
    return data


def louvain_step(G, copy, is_first_passage):
    # Step 1: Initialization, start with each node being a single community
    communities = {idx: set([node]) for idx, node in enumerate(G.nodes)}
    # To get direct access to the community (it speeds up a bit the algorithm)
    get_community = {node: idx for idx, node in enumerate(G.nodes)}
    # Used in the modularity computation
    neighbors_sets = {node: set(G.neighbors(node)) for node in G.nodes}
    m = len(G.edges)

    # Sum the weights of the incident edges for all nodes inside a community, for all communities
    # Separate first passage and other ones to speed up the algorithm
    if is_first_passage:
        sum_communities = {idx: sum(dict(G.degree(community)).values()) for idx, community in communities.items()}
    else:
        sum_communities = {idx: sum(dict(G.degree(community, 'weight')).values()) for idx, community in communities.items()}

    # Fixed number of iterations
    # TODO: change it?
    for i in range(4):
        print(f"Iteration {i}", flush=True)
        for node in tqdm(G.nodes):
            # Step 2: Remove node from its community
            # TODO: make sure what we want to do if the node has no neighboring communities
            neighboring_communities = get_neighboring_communities(G, node, get_community)
            if neighboring_communities == set():
                continue
            belong_to = get_community[node]
            communities[belong_to].remove(node)
            sum_communities[belong_to] -= G.degree(node, 'weight')
            if communities[belong_to] == set():
                del communities[belong_to]
                del sum_communities[belong_to]

            # Step 3: Insert the node in the community that maximizes the modularity
            scores = [
                (neighbor_community, modularity_gain(G, node, communities[neighbor_community], sum_communities[neighbor_community], neighbors_sets[node], m, is_first_passage))
                for neighbor_community in neighboring_communities
            ]
            best_community, best_score = max(scores, key=lambda x: x[1])
            communities[best_community].add(node)
            get_community[node] = best_community
            sum_communities[best_community] += G.degree(node, 'weight')

    if is_first_passage:
        return G, get_new_graph(G, communities, sum_communities, get_community), communities, get_community
    return G, get_new_graph(G, communities, sum_communities, get_community), communities, get_community


def get_neighboring_communities(G, node, get_community):
    # Use a set to make sure a community only appear once
    neighboring_communities = set()
    for neighbor in G.neighbors(node):
        if neighbor == node: continue
        neighboring_communities.add(get_community[neighbor])

    return neighboring_communities


def modularity_gain(G, node, community, sum_community, neighbor_set, m, is_first_passage):
    # Separate first passage and other ones to speed up the algorithm
    if is_first_passage:
        # Sum the weights of the edges from node into community nodes
        # Using sets allow to use intersection()
        sum_weights_node = len(neighbor_set.intersection(community))
        right_member = (sum_community * G.degree[node]) / (2 * (m**2))
    else:
        # Sum the weights of the edges from node into community nodes
        sum_weights_node = sum([G.get_edge_data(node, member)['weight'] for member in G.neighbors(node) if member in community])
        right_member = (sum_community * G.degree(node, 'weight')) / (2 * (m**2))
    # Compute modularity
    left_member = sum_weights_node / (2 * m)
    return left_member - right_member


def get_new_graph(old_G, communities, sum_communities, get_community):
    print("Construct new graph", flush=True)
    G = nx.Graph()
    G.add_nodes_from(communities.keys())
    for community in communities:
        G.add_edge(community, community, weight=sum_communities[community])

    for source, dest, weight_dict in old_G.edges(data=True):
        community1 = get_community[source]
        community2 = get_community[dest]
        current_weight = G.get_edge_data(community1, community2, {'weight': 0})['weight']
        # TODO: is this correct?
        new_weight = current_weight + weight_dict['weight']
        G.add_edge(community1, community2, weight=new_weight)
    return G

In [17]:
results = []
for idx, G in enumerate(nx_graphs):
    print(f"Louvain on {idx+1}/{len(nx_graphs)}", flush=True)
    data = louvain(G, 2)
    results.append(data)

Louvain on 1/3
Passage 0
Iteration 0


100%|██████████| 63332/63332 [00:02<00:00, 27120.47it/s]

Iteration 1



100%|██████████| 63332/63332 [00:01<00:00, 34774.22it/s]

Iteration 2


...

Iteration 2



100%|██████████| 25/25 [00:00<00:00, 25109.58it/s]

Iteration 3



100%|██████████| 25/25 [00:00<?, ?it/s]

Construct new graph
There are 17 communities after passage 1


In [41]:
for idx, result in enumerate(results):

    get_communities = {}
    # loop over the result
    for i in range(len(result)):
        _, _, get_community0 = result[i]
        get_communities[i] = get_community0
    
    intermediate_results = []

    for i in range(len(get_communities)-1, -1, -1):
        if i == 0:
            break
        curr = get_communities[i]
        prev = get_communities[i-1]
        intermediate_result =  {node: curr[val] for node, val in prev.items()}
        intermediate_results.append(intermediate_result)
    
    commu = intermediate_results[-1]


    # last_passage = result[len(result)-1]
    # networkx_graph, communities_dict, get_community = last_passage

    communities_df = pd.DataFrame(commu.items(), columns=['Id', 'Community'])
    communities_df["Label"] = communities_df["Id"].map(lambda x: x)
    communities_df["Is_author"] = False

    communities_df.loc[communities_df['Id'].isin(edgelists[idx]['Target'].unique()), "Is_author"] = True

    communities_df.to_csv(f"analysis/communities_df_{idx}.csv", index=False)

In [23]:
# _, _, get_community0 = results[0][0]
# _, _, get_community1 = results[0][1]

# get_community_test = {node: get_community1[val] for node, val in get_community0.items()}
# get_community_test

## TODO:
- Sample by taking random author_id and take all edges connected to it
- Implement from scratch community detection: Louvain (faster than Girvan)
- Look at how to store the communities (edgelist or so)
- Look at the caracteristics of the huge communities: what is the Tweet, who is the author, what are the hashtags etc...
- Make profiles of users: press, random guy, famous guy, etc...
- Look at outliers
- Load communities to Gephi to vizualise them